## 模板: ```candle_plot.ipynb``` 的 "開始回測" 部分

In [29]:
import pandas as pd
import utils

import plotly.graph_objects as go 

### 策略: 16 MA 與 64 MA 交叉

In [30]:
pair = 'EUR_USD'
granularity = 'H1'
ma_list = [16, 64] # loop through this much MA

In [31]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [32]:
non_cols  = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

### 製作多條 MA

In [33]:
df_ma = df[ ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c'] ].copy()

In [34]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma['mid_c'].rolling(window=ma).mean()

In [35]:
df_ma.dropna(inplace=True)

In [36]:
df_ma.head(5)

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
63,2021-01-21T07:00:00.000000000Z,1.21310,1.21353,1.21178,1.21197,1.211770,1.211451
64,2021-01-21T08:00:00.000000000Z,1.21196,1.21374,1.21149,1.21320,1.211906,1.211538
65,2021-01-21T09:00:00.000000000Z,1.21319,1.21460,1.21274,1.21424,1.212129,1.211643
66,2021-01-21T10:00:00.000000000Z,1.21422,1.21472,1.21351,1.21392,1.212316,1.211745
67,2021-01-21T11:00:00.000000000Z,1.21393,1.21462,1.21325,1.21433,1.212542,1.211845


### 作圖:

In [37]:
df_plot = df_ma.iloc[-200:].copy()

In [38]:
fig = go.Figure() 

# candle stick:
fig.add_trace(go.Candlestick(
    x = df_plot.time,
    open = df_plot.mid_o,
    high = df_plot.mid_h,
    low = df_plot.mid_l,
    close = df_plot.mid_c,

    line=dict(width=1),
    opacity=1,

    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))


# MA:
for ma in ma_list:
    col = f'MA_{ma}'

    fig.add_trace(go.Scatter(x=df_plot.time, 
    y=df_plot[col],
    line=dict(width=2),
    line_shape='spline', 
    name=f'MA_{ma}'
    ))


fig.update_layout(width=1000, height=400, 
    margin=dict(l=10, r=10, b=10, t=10), # 讓線圖更明顯
    font=dict(size=10, color='#e1e1e1'), # 將 x 軸資料轉為白色
    paper_bgcolor='#1e1e1e', # 將背景改為黑色
    plot_bgcolor='#1e1e1e') 

fig.update_xaxes( # 更新 x 軸資料
    gridcolor='#1f292f',
    showgrid=True, 
    fixedrange=True,
    rangeslider=dict(visible=False)
)

fig.update_yaxes( # 更新 y 軸資料
    gridcolor='#1f292f',
    showgrid=True, 
    fixedrange=True
)




In [39]:
df_ma.head(5)

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
63,2021-01-21T07:00:00.000000000Z,1.21310,1.21353,1.21178,1.21197,1.211770,1.211451
64,2021-01-21T08:00:00.000000000Z,1.21196,1.21374,1.21149,1.21320,1.211906,1.211538
65,2021-01-21T09:00:00.000000000Z,1.21319,1.21460,1.21274,1.21424,1.212129,1.211643
66,2021-01-21T10:00:00.000000000Z,1.21422,1.21472,1.21351,1.21392,1.212316,1.211745
67,2021-01-21T11:00:00.000000000Z,1.21393,1.21462,1.21325,1.21433,1.212542,1.211845


### 將策略用 code 寫出

In [40]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [41]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF
63,2021-01-21T07:00:00.000000000Z,1.21310,1.21353,1.21178,1.21197,1.211770,1.211451,0.000319
64,2021-01-21T08:00:00.000000000Z,1.21196,1.21374,1.21149,1.21320,1.211906,1.211538,0.000368
65,2021-01-21T09:00:00.000000000Z,1.21319,1.21460,1.21274,1.21424,1.212129,1.211643,0.000485
66,2021-01-21T10:00:00.000000000Z,1.21422,1.21472,1.21351,1.21392,1.212316,1.211745,0.000571
67,2021-01-21T11:00:00.000000000Z,1.21393,1.21462,1.21325,1.21433,1.212542,1.211845,0.000697


#### 但這個策略不是只計算出 MA 的差距
#### 而是在這個差距由 "正轉負" 或 "負轉正" 時進場  
#### 換句話說，我們還要算 "今日 DIFF" 與 "昨日 DIFF" 的差距

In [42]:
# DIFF_PREV = 昨日 DIFF
df_ma['DIFF_PREV'] = df_ma['DIFF'].shift(1)

In [43]:
df_ma.head() # note that there's no value at first row

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV
63,2021-01-21T07:00:00.000000000Z,1.21310,1.21353,1.21178,1.21197,1.211770,1.211451,0.000319,NaN
64,2021-01-21T08:00:00.000000000Z,1.21196,1.21374,1.21149,1.21320,1.211906,1.211538,0.000368,0.000319
65,2021-01-21T09:00:00.000000000Z,1.21319,1.21460,1.21274,1.21424,1.212129,1.211643,0.000485,0.000368
66,2021-01-21T10:00:00.000000000Z,1.21422,1.21472,1.21351,1.21392,1.212316,1.211745,0.000571,0.000485
67,2021-01-21T11:00:00.000000000Z,1.21393,1.21462,1.21325,1.21433,1.212542,1.211845,0.000697,0.000571


In [44]:
def is_trade(row): # this function should be executed row by row
    if row.DIFF >= 0 and row.DIFF_PREV < 0: # 進場做多
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0: # 進場做空
        return True 
    else:  # 否則就不進場
        return False

### 辨認是否交易

In [45]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1) # apply function row by row

In [46]:
df_trades = df_ma[df_ma['IS_TRADE'] == True].copy()

In [47]:
df_trades.head() # 可以發現每個 row 的 DIFF 與 DIFF_PREV 均正負相間

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
121,2021-01-25T17:00:00.000000000Z,1.21368,1.21475,1.21362,1.21445,1.215794,1.215855,-0.000060,0.000096,True
153,2021-01-27T01:00:00.000000000Z,1.21624,1.21672,1.21584,1.21610,1.215634,1.215497,0.000137,-0.000083,True
164,2021-01-27T12:00:00.000000000Z,1.21164,1.21198,1.21056,1.21079,1.214941,1.215047,-0.000106,0.000171,True
216,2021-01-29T16:00:00.000000000Z,1.21492,1.21534,1.21298,1.21352,1.212039,1.211844,0.000195,-0.000022,True
236,2021-02-01T12:00:00.000000000Z,1.20818,1.20887,1.20722,1.20756,1.211494,1.211547,-0.000052,0.000272,True


In [48]:
df_trades.shape # 有 85 次投資機會

(85, 10)

### 將第一次投資機會畫出來

In [49]:
# 第一次投資機會出現在第 121 row
# 注意到交易時間為 2021.01.25 17:00 UTC
df_plot = df_ma.iloc[120:170].copy()

In [50]:
fig = go.Figure() 

# candle stick:
fig.add_trace(go.Candlestick(
    x = df_plot.time,
    open = df_plot.mid_o,
    high = df_plot.mid_h,
    low = df_plot.mid_l,
    close = df_plot.mid_c,

    line=dict(width=1),
    opacity=1,

    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))


# MA:
for ma in ma_list:
    col = f'MA_{ma}'

    fig.add_trace(go.Scatter(x=df_plot.time, 
    y=df_plot[col],
    line=dict(width=2),
    line_shape='spline', 
    name=f'MA_{ma}'
    ))


fig.update_layout(width=1000, height=400, 
    margin=dict(l=10, r=10, b=10, t=10), # 讓線圖更明顯
    font=dict(size=10, color='#e1e1e1'), # 將 x 軸資料轉為白色
    paper_bgcolor='#1e1e1e', # 將背景改為黑色
    plot_bgcolor='#1e1e1e') 

fig.update_xaxes( # 更新 x 軸資料
    gridcolor='#1f292f',
    showgrid=True, 
    fixedrange=True,
    rangeslider=dict(visible=False)
)

fig.update_yaxes( # 更新 y 軸資料
    gridcolor='#1f292f',
    showgrid=True, 
    fixedrange=True
)

剛才看 data frame 時，首次交易機會在 2021.01.25 17:00 UTC，  
但實際畫圖發現，MA 交叉點在 2021.01.29 15:00 UTC。  
---
會這樣是因為，我們一開始把 NaN 值都去掉了，因此 Data frame 的編號跟實際發生日期的編號不一致

### 重設 data frame index

In [51]:
df_ma.reset_index(drop=True, inplace=True)

In [52]:
df_trades = df_ma[df_ma['IS_TRADE'] == True].copy()

In [53]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
58,2021-01-25T17:00:00.000000000Z,1.21368,1.21475,1.21362,1.21445,1.215794,1.215855,-0.000060,0.000096,True
90,2021-01-27T01:00:00.000000000Z,1.21624,1.21672,1.21584,1.21610,1.215634,1.215497,0.000137,-0.000083,True
101,2021-01-27T12:00:00.000000000Z,1.21164,1.21198,1.21056,1.21079,1.214941,1.215047,-0.000106,0.000171,True
153,2021-01-29T16:00:00.000000000Z,1.21492,1.21534,1.21298,1.21352,1.212039,1.211844,0.000195,-0.000022,True
173,2021-02-01T12:00:00.000000000Z,1.20818,1.20887,1.20722,1.20756,1.211494,1.211547,-0.000052,0.000272,True


In [54]:
df_plot = df_ma.iloc[50:100].copy() # 注意到這邊是用 df_ma 來取值，不是 df_trades !!!

In [55]:
fig = go.Figure() 

# candle stick:
fig.add_trace(go.Candlestick(
    x = df_plot.time,
    open = df_plot.mid_o,
    high = df_plot.mid_h,
    low = df_plot.mid_l,
    close = df_plot.mid_c,

    line=dict(width=1),
    opacity=1,

    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))


# MA:
for ma in ma_list:
    col = f'MA_{ma}'

    fig.add_trace(go.Scatter(x=df_plot.time, 
    y=df_plot[col],
    line=dict(width=2),
    line_shape='spline', 
    name=f'MA_{ma}'
    ))


fig.update_layout(width=1000, height=400, 
    margin=dict(l=10, r=10, b=10, t=10), # 讓線圖更明顯
    font=dict(size=10, color='#e1e1e1'), # 將 x 軸資料轉為白色
    paper_bgcolor='#1e1e1e', # 將背景改為黑色
    plot_bgcolor='#1e1e1e') 

fig.update_xaxes( # 更新 x 軸資料
    gridcolor='#1f292f',
    showgrid=True, 
    fixedrange=True,
    rangeslider=dict(visible=False)
)

fig.update_yaxes( # 更新 y 軸資料
    gridcolor='#1f292f',
    showgrid=True, 
    fixedrange=True
)